## Code to work with ATM data along the flowline of Zachariae Isstrom

**by Jukes Liu**

**June 2019 ICESat-2 Hackweek**


In [2]:
#IMPORT PACKAGES
import os
import glob
import pandas as pd
import csv
import numpy as np
import math
import matplotlib.pyplot as plt
import pyproj
from scipy.spatial import distance

Plot the elevation profile along the flowline:

In [6]:
%matplotlib widget

plt.plot(dist_along, ATM_elev)
plt.ylabel('Elevation (m)')
plt.xlabel('distance along flow (km)')
plt.title('Elevation along flowline of Zachariae Isstrom')
plt.show()

FigureCanvasNbAgg()

Plot the horizontal footprint with elevation color coded:

In [7]:
%matplotlib widget

plt.scatter(ATM_long, ATM_lat, c=ATM_elev,cmap='plasma')
plt.ylabel('Latitude (deg)')
plt.xlabel('Longitude (deg)')
plt.title('ATM flyover along Zachariae Isstrom')
plt.colorbar(fraction=0.0320, pad=0.02, label='Elevation (m)')
plt.show()

FigureCanvasNbAgg()

Final data extracted into arrays:
+ ATM_long: longitude of elev points
+ ATM_lat: latitude of elev points
+ PS_x: x-coords of elev points in Greenland Polar Stereo
+ PS_y: y-coords of elev points in Greenland Polar Stereo
+ ATM_elev: elevation points with z in meters
+ dist_along: distance along the flowline in km

...and recombined into a dataframe:

In [8]:
variables = [ATM_long, ATM_lat, PS_x, PS_y, ATM_elev, dist_along]
indices = ['ATM_long', "ATM_lat", "PS_x", "PS_y", "ATM_elev", "dist_along"]

#create DataFrame from these variables
final_data = pd.DataFrame(variables)
final_data = final_data.transpose()
final_data.columns = indices
print(final_data.head())

     ATM_long    ATM_lat           PS_x          PS_y  ATM_elev  dist_along
0  334.846721  79.211271  397913.477068 -1.102426e+06  891.4235    0.000000
1  334.848221  79.211989  397915.704090 -1.102342e+06  891.6056   84.231835
2  334.849722  79.212707  397917.946244 -1.102257e+06  891.7156  168.470291
3  334.848567  79.212155  397916.203061 -1.102322e+06  891.6397  233.240595
4  334.848296  79.211153  397948.158670 -1.102427e+06  891.1921  342.853729


Get target points from ATM csv and put into dataframe

In [14]:
ATM = pd.read_csv(file)


tpoints = final_data[['PS_x','PS_y']].copy()
tpoints = tpoints.values


# print(tpoints[0:5])




[[  397913.47706814 -1102425.77908309]
 [  397915.70409007 -1102341.5766936 ]
 [  397917.94624422 -1102257.36808201]
 [  397916.20306121 -1102322.11492441]
 [  397948.15866962 -1102426.96662082]]


(13921, 2)

Get query points from ATL06 csv and put into dataframe

In [100]:
ATL06 = pd.read_csv(file)

qpoints = ATL06[['x','y']].copy()
qpoints = qpoints.values


[[ 79.4 -23.1]
 [ 79.2 -22.6]
 [ 79.1 -22.4]
 [ 79.  -22.1]
 [ 78.8 -21.6]
 [ 78.6 -21.1]]
<class 'numpy.ndarray'>
(6, 2)


In [30]:
Intersections = {'Flow_dist_ATM':[],'z_ATM':[],'idx_ATM':[],'z_ATL06':[],'t_ATL06':[],'idx_ATL06':[]}
Intersections = pd.DataFrame(data = Intersections)

Intersections.head()

,Flow_dist_ATM,idx_ATM,z_ATM,z_ATL06,t_ATL06,idx_ATL06


In [104]:
def closest_node(node, nodes):
    closest_index = distance.cdist([node], nodes).argmin()
    closest_dist = np.min(distance.cdist([node], nodes,'euclidean'))
    return closest_index, closest_dist

close_idx = []
min_dist = []
for i in range(len(tpoints)):
    close_idxT,min_distT = closest_node(tpoints[i,:], qpoints)
    close_idx = np.append(close_idx,[close_idxT])
    min_dist = np.append(min_dist,[min_distT])

tpt_idx = np.argmin(min_dist)
qpt_idx = int(close_idx[tpt_idx])
print(tpt_idx)
print(qpt_idx)

AddDatarow = ['Flow_dist_ATM':[ATMdata.loc[tpt_idx,'dist_along']],
       'z_ATM':[ATMdata.loc[tpt_idx,'ATM_elev']],
       'idx_ATM':[tpt_idx],
       'z_ATL06':[ATL06data.loc[qpt_idx,'z']],
       't_ATL06':[ATL06data.loc[qpt_idx,'t']],
       'idx_ATL06':[qpt_idx]]

1
2


In [52]:
Intersections.append(AddDatarow)


Intersections.to_csv(index=False)